### Library ###

In [1]:
!pip install -q bitsandbytes trl datasets accelerate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0

In [2]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, BitsAndBytesConfig, TrainingArguments, DataCollatorForLanguageModeling
from transformers import pipeline
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, PeftConfig
from trl import SFTTrainer
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import statistics

In [3]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

#### Pre Trained Model ####

In [4]:
model_name = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
#"NousResearch/Llama-2-7b-chat-hf"

##### Quantize #####

In [5]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

#### Model Load ####

In [6]:
device_map = {"": 0}

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# pip install -U bitsandbytes

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

#### Dealing Dataset ####

In [9]:
datasets = load_dataset("SAGI-1/reasoningData_200k")

README.md:   0%|          | 0.00/421 [00:00<?, ?B/s]

(…)-00000-of-00001-e32f5c3d72ded71d.parquet:   0%|          | 0.00/125M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/201928 [00:00<?, ? examples/s]

In [10]:
# Taking a sample of data
dataset = datasets['train'].shuffle(seed=42).select(range(2468))

In [11]:
test_modelwithData = datasets['train'].shuffle(seed=29).select(range(100))

In [12]:
def clean_text(text):
    # Remove unwanted characters
    text = re.sub(r"[\n\r\-_(){}\[\]\\/]", " ", text)
    text = re.sub(r"\s+", " ", text)  # remove extra spaces
    return text.strip()

In [ ]:
# def truncate_instruction(text):
#     # Keep only the part before the first stop punctuation
#     text = clean_text(text)
#     stop_match = re.split(r"[.,!?]", text, maxsplit=1)
#     return stop_match[0].strip()

In [13]:
def preprocess_example(example):
    return {
        "subject": clean_text(example["instruction"]),
        "email": clean_text(example["answer"])
    }

cleaned_dataset = dataset.map(preprocess_example, remove_columns=["instruction", "answer"])

Map:   0%|          | 0/2468 [00:00<?, ? examples/s]

In [14]:
def convert_to_instruction_format(example):
    return {
        "text": f"<s>[INST] {example['email']} [/INST] {example['subject']} </s>"
    }

formatted_dataset = cleaned_dataset.map(convert_to_instruction_format)
formatted_dataset = formatted_dataset.remove_columns(['subject', 'email'])

Map:   0%|          | 0/2468 [00:00<?, ? examples/s]

In [15]:
new_model = "llama-subjectline"

### LoRA config & Train ###

In [16]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

In [17]:
training_args = TrainingArguments(
    output_dir="./llama-subject-finetuned",
    num_train_epochs=5,
    per_device_train_batch_size=6,
    gradient_accumulation_steps=1,
    save_steps = 0,
    optim="paged_adamw_32bit",
    logging_steps=20,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio = 0.03,
    group_by_length = True,
    lr_scheduler_type="cosine",
    report_to="tensorboard"
)

In [18]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=lora_config,
    args=training_args,
)

Converting train dataset to ChatML:   0%|          | 0/2468 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/2468 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2468 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2909 > 2048). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/2468 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [19]:
trainer.train()

Step,Training Loss
20,2.286800
40,2.563000
60,2.387100
80,2.202800
100,2.047300
120,2.083100
140,2.070500
160,2.005100
180,2.042900
200,1.962600


TrainOutput(global_step=2060, training_loss=1.8063689620749463, metrics={'train_runtime': 1012.0541, 'train_samples_per_second': 12.193, 'train_steps_per_second': 2.035, 'total_flos': 1.9992926783176704e+16, 'train_loss': 1.8063689620749463})

#### Load and Test New Model ####

In [20]:
trainer.model.save_pretrained(new_model)

In [21]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)

Device set to use cuda:0


In [25]:
def test_model(email):
    result = pipe(f"<s>[INST] {email} [/INST]")
    result_text = result[0]['generated_text']

    if "[INST]" in result_text and "[/INST]" in result_text:
        subject = result_text.split("[/INST]")[-1].strip()
        return f"Email:\n{email}\n\nSubject:\n{subject}"

    else:
      return f"Subject Can not be generated for given Email body: {email}"

In [24]:
email = 'The idiom "it’s raining cats and dogs" means that it is raining heavily'
print(test_model(email))

Email:
The idiom "it’s raining cats and dogs" means that it is raining heavily

Subject:
What is the idiom "it’s raining cats and dogs"?


In [ ]:
formatted_dataset[10]

{'text': '<s>[INST] The idiom "it’s raining cats and dogs" means that it is raining heavily. [/INST] What is the meaning of the following idiom? </s>'}

#### Evaluation ####

In [26]:
def pred_sub(email):
    result = pipe(f"<s>[INST] {email} [/INST]")
    result_text = result[0]['generated_text']

    if "[INST]" in result_text and "[/INST]" in result_text:
        subject = result_text.split("[/INST]")[-1].strip()
        return subject

In [27]:
cleaned_dataset_test = test_modelwithData.map(preprocess_example, remove_columns=["instruction", "answer"])

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [30]:
bleu_scores = []
rouge_scores = []
meteor_scores = []

for i in range(len(cleaned_dataset_test)):
  reference = cleaned_dataset_test[i]['subject']
  candidate = pred_sub(cleaned_dataset_test[i]['email'])

  print(f"{i+1} \nReference: {reference}\nCandidate: {candidate}\n")
  smoothie = SmoothingFunction().method4
  bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothie)
  bleu_scores.append(bleu_score)

  scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
  rge_scores = scorer.score(reference, candidate)
  rouge_scores.append(rge_scores['rougeL'].fmeasure)

  mt_scores = meteor_score([word_tokenize(reference)], word_tokenize(candidate))
  meteor_scores.append(mt_scores)

  print(f"BLEU Score: {bleu_score:.4f}")
  print(f"ROUGE Score: {rge_scores['rougeL'].fmeasure:.4f}")
  print(f"METEOR Score: {mt_scores:.4f}\n\n")

1 
Reference: What is the difference between an exothermic and endothermic reaction?
Candidate: What is the difference between exothermic and endothermic reactions?

BLEU Score: 0.9416
ROUGE Score: 0.9474
METEOR Score: 0.9138


2 
Reference: How can you get more song birds to visit your yard? put out some hamburgers to attract them adopt some large dogs to stay in the yard call Here birdy birdy! out the window hang an open box containing tasty seeds from a tree Which is the correct answer?
Candidate: What would attract song birds to your yard?

BLEU Score: 0.0063
ROUGE Score: 0.1754
METEOR Score: 0.1005


3 
Reference: In this task, you are given a text from a social media post. Your task is to classify the given post into two categories: 1 yes if the given post is sexually offensive, 2 no, otherwise. Emphasis on sexually offensive or any lewd reference. Generate label 'no' for offensive posts that do not reference sexually explicit content. Warning: the examples and instances may cont

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


7 
Reference: Given a passage, construct a question on the basis of the information present in the passage. Construct the question in such a way that i it is unambiguous, ii it is answerable from the passage, iii its answer is unique iv it answer uses text phrases from the passage. Avoid creating questions that i can't be answered correctly without actually understanding the passage and ii uses the same words or phrases given in the passage. Calcitonin was recently reported to counter progression of cartilage degradation in an experimental model of osteoarthritis, and the effects were primarily suggested to be mediated by inhibition of subchondral bone resorption. We investigated direct effects of calcitonin on chondrocytes by assessing expression of the receptor and pharmacological effects on collagen type II degradation under ex vivo and in vivo conditions.', 'Localization of the calcitonin receptor on articular chondrocytes was investigated by immunohistochemistry, and the expressio

In [31]:
print(f"##### Final Scores #####")
print(f"Avg. BLEU Score: {statistics.mean(bleu_scores):.4f}")
print(f"Avg. ROUGE Score: {statistics.mean(rouge_scores):.4f}")
print(f"Avg. METEOR Score: {statistics.mean(meteor_scores):.4f}")

##### Final Scores #####
Avg. BLEU Score: 0.3230
Avg. ROUGE Score: 0.3034
Avg. METEOR Score: 0.3136
